<br>

# Introdução

In [ ]:
import os
import pprint
import pandas as pd
from collections import OrderedDict

In [ ]:
def get_parameters():
    # Read Data
    try:
        #print('Read local table')
        df_357 = pd.read_excel(
            io=os.path.join(os.path.dirname(__file__), 'data', 'tab_conama_357.xlsx'),
            sheet_name='conama_357',
            index_col=0,
        )
    except Exception as e:
        #print(e, '\n')
        #print('Read table from GitHub')
        df_357 = pd.read_excel(
            io='https://raw.githubusercontent.com/gaemapiracicaba/norma_res_conama_357-05/main/src/normas/data/tab_conama_357.xlsx',
            sheet_name='conama_357',
            index_col=0,
        )

    # Filter only quality
    df_357 = df_357.loc[(df_357['tipo_padrao'] == 'qualidade')]
    #print(df_357.head())

    # Classes
    list_classes = list(set(df_357['padrao_qualidade']))
    list_classes = [x for x in list_classes if pd.notnull(x)]
    list_classes.sort()    

    return df_357, list_classes

In [ ]:
df_357, list_classes = get_parameters()
df_357.head()
pprint.pprint(list_classes)

In [ ]:
def filter_by_classe(df_357, classe):
    # Filter dataframe by Classe
    df_357 = df_357.loc[(df_357['padrao_qualidade'] == classe)]

    # Parâmetros
    list_parametros = list(set(df_357['parametro_descricao']))
    list_parametros = [x for x in list_parametros if pd.notnull(x)]
    list_parametros.sort()
    return df_357, list_parametros

In [ ]:
df_357, list_parametros = filter_by_classe(df_357, classe='Classe 2')
pprint.pprint(list_parametros)

In [ ]:
def filter_by_parameters(df_357, parametro, condicao=None):
    # Filter dataframe by Parametro
    df_357 = df_357.loc[(df_357['parametro_descricao'] == parametro)]

    # Condição
    array = df_357['condicao'].values
    dict_condicao = dict(enumerate(array.flatten(), 1))

    # Check and Get Results
    if len(df_357) == 1 and len(array) == 1:
        dict_357 = df_357.to_dict(orient='records')[0]
        dict_357 = OrderedDict(sorted(dict_357.items(), key=lambda x: df_357.columns.get_loc(x[0])))
        return dict_357

    elif len(df_357) > 1 and len(array) > 1 and condicao is not None:
        try:
            # Filtra a Condição
            #condicao = df_357['condicao'].values[condicao]
            df_357 = df_357.loc[(df_357['condicao'] == dict_condicao[int(condicao)])]
            dict_357 = df_357.to_dict(orient='records')[0]
            dict_357 = OrderedDict(sorted(dict_357.items(), key=lambda x: df_357.columns.get_loc(x[0])))
            return dict_357
        except Exception as e:
            #print(e)
            print(f'A condição definida foi "{condicao}".\nAs opções possíveis são:\n')
            print(*('{} - {}'.format(k, v) for k,v in dict_condicao.items()), sep='\n')

    else:
        print(f'Parâmetro "{parametro}" tem mais de um registro.\nFaz-se necessário definir condição!\n')
        print(*('{} - {}'.format(k, v) for k,v in dict_condicao.items()), sep='\n')

In [ ]:
# Filter Data by Parâmetros
#dict_357 = filter_by_parameters(df_357, parametro='Cobre Dissolvido')
dict_357 = filter_by_parameters(df_357, parametro='Nitrogênio Amoniacal Total')
#dict_357 = filter_by_parameters(df_357, parametro='Fósforo Total')
#dict_357 = filter_by_parameters(df_357, parametro='Fósforo Total', condicao=3)
dict_357

In [ ]:
def set_type_desconformidade(dict_357):
    if pd.isnull(dict_357['valor_minimo_permitido']) & pd.notnull(dict_357['valor_maximo_permitido']):
        #print('Parâmetro só tem "valor máximo". Caso o valor medido esteja acima, é amostra desconforme!')
        tipo_357 = 'acima>desconforme'

    elif pd.notnull(dict_357['valor_minimo_permitido']) & pd.isnull(dict_357['valor_maximo_permitido']):
        #print('Parâmetro só tem "valor mínimo". Caso o valor medido esteja abaixo, é amostra desconforme!')
        tipo_357 = 'abaixo>desconforme'

    elif pd.notnull(dict_357['valor_minimo_permitido']) & pd.notnull(dict_357['valor_maximo_permitido']):
        #print('Parâmetro tem "valor mínimo" e "valor máximo". Caso o valor medido acima ou abaixo, é amostra desconforme!')
        tipo_357 = 'abaixo_acima>desconforme'

    elif pd.isnull(dict_357['valor_minimo_permitido']) & pd.isnull(dict_357['valor_maximo_permitido']):
        #print('Erro!')
        tipo_357 = 'erro'
    else:
        print('Erro!')
        tipo_357 = 'erro'

    return tipo_357

In [ ]:
set_type_desconformidade(dict_357)

In [ ]:
def evaluate_result(valor, dict_357):
    # Get type
    tipo_357 = set_type_desconformidade(dict_357)

    # Evaluate type
    if tipo_357 == 'acima>desconforme':
        if valor > dict_357['valor_maximo_permitido']:
            result_357 = 'desconforme'
        else:
            result_357 = 'conforme'

    elif tipo_357 == 'abaixo>desconforme':
        if valor < dict_357['valor_minimo_permitido']:
            result_357 = 'desconforme'
        else:
            result_357 = 'conforme'

    elif tipo_357 == 'abaixo_acima>desconforme':
        if dict_357['valor_minimo_permitido'] <= valor <= dict_357['valor_maximo_permitido']:
            result_357 = 'conforme'
        else:
            result_357 = 'desconforme'

    elif tipo_357 == 'erro':
        result_357 = 'erro'

    else:
        result_357 = 'erro'

    return result_357

In [ ]:
valor = 10
evaluate_result(valor, dict_357)

<br>

# Export

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
input_filename = 'conama_357.ipynb'
input_filepath = os.path.join(os.getcwd(), input_filename)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(input_filepath)

# Write to output html file
with open(os.path.join(os.getcwd(), '..', 'src', 'normas', 'conama_357.py'),  'w', encoding='utf-8') as f:
    f.write(body)